In [12]:
# EXERCISE: 2
# SUBMITTED BY: QUINCY RODGE A. MACALALAG BSCS 3A AI
# DATE: SEP 12, 2025

import numpy as np

# -------------------------
# Dense_Layer class 
# -------------------------
class Dense_Layer:
    def __init__(self, input_size=None, output_size=None, weights=None, bias=None):
        if weights is not None:
            self.weights = np.array(weights)
            input_size, output_size = self.weights.shape
        else:
            if input_size is None or output_size is None:
                raise ValueError("If weights are not provided, input_size and output_size must be specified.")
            self.weights = np.random.randn(input_size, output_size)
        
        self.bias = np.array(bias) if bias is not None else np.random.randn(output_size)
        self.inputs = None
        self.z = None
        self.output = None

    def set_inputs(self, inputs):
        self.inputs = np.array(inputs)

    def forward(self):
        self.z = np.dot(self.inputs, self.weights) + self.bias
        return self.z

    def activate(self, activation="relu"):
        if self.z is None:
            self.forward()

        if activation == "relu":
            self.output = np.maximum(0, self.z)
        elif activation == "sigmoid":
            self.output = 1 / (1 + np.exp(-self.z))
        elif activation == "softmax":
            exp_vals = np.exp(self.z - np.max(self.z))
            self.output = exp_vals / np.sum(exp_vals)
        else:
            raise ValueError("Unsupported activation function. Use: relu, sigmoid, softmax.")
        return self.output

    def loss(self, target_output):
        self.target = np.array(target_output)
        self.loss_value = np.mean((self.output - self.target) ** 2)
        return self.loss_value

# --------------------------------------
# Helper: ensure weight orientation is (input_size, output_size)
# --------------------------------------
def arrange_weights(weights, bias, inputs, layer_name=""):
    w = np.array(weights)
    b = np.array(bias)
    ins = np.array(inputs)
    # (input_size, output_size)
    desired_shape = (ins.shape[1], len(b))   
    if w.shape == desired_shape:
        return w, False
    if w.T.shape == desired_shape:
        return w.T, True
        
    # If neither matches, try one more inference attempt:
    # If weights shape is (output_size, input_size) transpose.
    if w.shape[0] == len(b) and w.shape[1] == ins.shape[1]:
        return w.T, True
    raise ValueError(f"Cannot reconcile weights shape {w.shape} with inputs shape {ins.shape} and bias length {len(b)} for {layer_name}.")



In [13]:
# ===============================
# Problem 1 – Iris Dataset Classification
# ===============================

# Inputs & target
X = [[5.1, 3.5, 1.4, 0.2]]
target_output = [[0.7, 0.2, 0.1]]

# First hidden layer
weights_1 = [
    [0.2, 0.5, -0.3],
    [0.1, -0.2, 0.4],
    [-0.4, 0.3, 0.2],
    [0.6, -0.1, 0.5],   
]
b_1 = [3.0, -2.1, 0.6]

# Second hidden layer
weights_2 = [
    [0.3, -0.5],
    [0.7, 0.2],
    [-0.6, 0.4]
]
b_2 = [4.3, 6.4]

# Last hidden layer
weights_3 = [
    [0.5, -0.3, 0.8],
    [-0.2, 0.6, -0.4]
]
b_3 = [-1.5, 2.1, -3.3]

print("Problem 1 – Iris Dataset Classification\n")
print("Given Input (X):", X)
print("Target Output :", target_output)
print("-" * 50)

# ----- Layer 1 -----
layer1 = Dense_Layer(weights=weights_1, bias=b_1)
layer1.set_inputs(X)
z1 = layer1.forward()
a1 = layer1.activate("relu")

print("\nLayer 1 (First Hidden Layer)")
print("Inputs:", X)
print("Weights:\n", np.array(weights_1))
print("Bias:", b_1)
print("Weighted Sum (Z1):", np.round(z1, 3))
print("Activation (ReLU):", np.round(a1, 3))

# ----- Layer 2 -----
layer2 = Dense_Layer(weights=weights_2, bias=b_2)
layer2.set_inputs(a1)
z2 = layer2.forward()
a2 = layer2.activate("sigmoid")

print("\nLayer 2 (Second Hidden Layer)")
print("Inputs:", np.round(a1, 3))
print("Weights:\n", np.array(weights_2))
print("Bias:", b_2)
print("Weighted Sum (Z2):", np.round(z2, 3))
print("Activation (Sigmoid):", np.round(a2, 3))

# ----- Layer 3 -----
layer3 = Dense_Layer(weights=weights_3, bias=b_3)
layer3.set_inputs(a2)
z3 = layer3.forward()
a3 = layer3.activate("softmax")

print("\nLayer 3 (Output Layer)")
print("Inputs:", np.round(a2, 3))
print("Weights:\n", np.array(weights_3))
print("Bias:", b_3)
print("Weighted Sum (Z3):", np.round(z3, 3))
print("Activation (Softmax):", np.round(a3, 3))

# ----- Final Result -----
loss_val = layer3.loss(target_output)
predicted_class = np.argmax(a3)

classes = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]

print("\n" + "=" * 50)
print("Final Classification Result")
print("Predicted Output (Softmax):", np.round(a3, 3))
print("Target Output            :", target_output)
print("Loss (MSE)               :", round(loss_val, 3))
print("Predicted Class          :", classes[predicted_class])


Problem 1 – Iris Dataset Classification

Given Input (X): [[5.1, 3.5, 1.4, 0.2]]
Target Output : [[0.7, 0.2, 0.1]]
--------------------------------------------------

Layer 1 (First Hidden Layer)
Inputs: [[5.1, 3.5, 1.4, 0.2]]
Weights:
 [[ 0.2  0.5 -0.3]
 [ 0.1 -0.2  0.4]
 [-0.4  0.3  0.2]
 [ 0.6 -0.1  0.5]]
Bias: [3.0, -2.1, 0.6]
Weighted Sum (Z1): [[3.93 0.15 0.85]]
Activation (ReLU): [[3.93 0.15 0.85]]

Layer 2 (Second Hidden Layer)
Inputs: [[3.93 0.15 0.85]]
Weights:
 [[ 0.3 -0.5]
 [ 0.7  0.2]
 [-0.6  0.4]]
Bias: [4.3, 6.4]
Weighted Sum (Z2): [[5.074 4.805]]
Activation (Sigmoid): [[0.994 0.992]]

Layer 3 (Output Layer)
Inputs: [[0.994 0.992]]
Weights:
 [[ 0.5 -0.3  0.8]
 [-0.2  0.6 -0.4]]
Bias: [-1.5, 2.1, -3.3]
Weighted Sum (Z3): [[-1.201  2.397 -2.902]]
Activation (Softmax): [[0.027 0.969 0.005]]

Final Classification Result
Predicted Output (Softmax): [[0.027 0.969 0.005]]
Target Output            : [[0.7, 0.2, 0.1]]
Loss (MSE)               : 0.351
Predicted Class          : Ir

In [14]:

# ===============================
# Problem 2 – Breast Cancer Classification
# ===============================

# Input and target
X = [[14.1, 20.3, 0.095]]         # 1 x 3
target_output = [[1]]            # 1 x 1

# First hidden layer (3 inputs -> 3 outputs)
weights_1 = [
    [0.5, -0.3, 0.8],
    [0.2, 0.4, -0.6],
    [-0.7, 0.9, 0.1],
]
b_1 = [0.3, -0.5, 0.6]

# Second hidden layer (expected 3 inputs -> 2 outputs)
weights_2 = [
    [0.6, -0.2, 0.4],
    [-0.3, 0.5, 0.7]
]
b_2 = [0.1, -0.8]

# Last layer (expected 2 inputs -> 1 output)
weights_3 = [
    [0.7, -0.5]
]
b_3 = [0.2]

print("Problem 2 – Breast Cancer Classification\n")
print("Given Input (X):", X)
print("Target Output :", target_output)
print("-" * 60)

# ----- Layer 1 -----
w1, t1 = arrange_weights(weights_1, b_1, X, layer_name="Layer 1")
layer1 = Dense_Layer(weights=w1, bias=b_1)
layer1.set_inputs(X)
z1 = layer1.forward()
a1 = layer1.activate("relu")

print("\nLayer 1 (First Hidden Layer)")
print("Inputs:", np.round(X, 3))
print("Weights (shape {}):\n{}".format(np.array(w1).shape, np.array(w1)))
print("Bias:", b_1)
print("Weighted Sum (Z1):", np.round(z1, 3))
print("Activation (ReLU):", np.round(a1, 3))
if t1:
    print("Note: weights_1 were transposed to match input shape.")

# ----- Layer 2 -----
w2, t2 = arrange_weights(weights_2, b_2, a1, layer_name="Layer 2")
layer2 = Dense_Layer(weights=w2, bias=b_2)
layer2.set_inputs(a1)
z2 = layer2.forward()
a2 = layer2.activate("sigmoid")

print("\nLayer 2 (Second Hidden Layer)")
print("Inputs:", np.round(a1, 3))
print("Weights (shape {}):\n{}".format(np.array(w2).shape, np.array(w2)))
print("Bias:", b_2)
print("Weighted Sum (Z2):", np.round(z2, 3))
print("Activation (Sigmoid):", np.round(a2, 6))
if t2:
    print("Note: weights_2 were transposed to match input shape.")

# ----- Layer 3 -----
w3, t3 = arrange_weights(weights_3, b_3, a2, layer_name="Layer 3")
layer3 = Dense_Layer(weights=w3, bias=b_3)
layer3.set_inputs(a2)
z3 = layer3.forward()
a3 = layer3.activate("sigmoid")

print("\nLayer 3 (Output Layer)")
print("Inputs:", np.round(a2, 6))
print("Weights (shape {}):\n{}".format(np.array(w3).shape, np.array(w3)))
print("Bias:", b_3)
print("Weighted Sum (Z3):", np.round(z3, 3))
print("Activation (Sigmoid):", np.round(a3, 6))
if t3:
    print("Note: weights_3 were transposed to match input shape.")

# ----- Final Result -----
loss_val = layer3.loss(target_output)
predicted_prob = float(a3.flatten()[0])
predicted_label = 1 if predicted_prob >= 0.5 else 0
label_name = "Malignant (1)" if predicted_label == 1 else "Benign (0)"

print("\n" + "=" * 60)
print("Final Classification Result")
print("Predicted Probability (sigmoid):", np.round(a3, 6))
print("Target Output               :", target_output)
print("Loss (MSE)                  :", round(loss_val, 6))
print("Predicted Label             :", predicted_label, "-", label_name)
print("=" * 60)

Problem 2 – Breast Cancer Classification

Given Input (X): [[14.1, 20.3, 0.095]]
Target Output : [[1]]
------------------------------------------------------------

Layer 1 (First Hidden Layer)
Inputs: [[14.1   20.3    0.095]]
Weights (shape (3, 3)):
[[ 0.5 -0.3  0.8]
 [ 0.2  0.4 -0.6]
 [-0.7  0.9  0.1]]
Bias: [0.3, -0.5, 0.6]
Weighted Sum (Z1): [[11.344  3.476 -0.29 ]]
Activation (ReLU): [[11.344  3.476  0.   ]]

Layer 2 (Second Hidden Layer)
Inputs: [[11.344  3.476  0.   ]]
Weights (shape (3, 2)):
[[ 0.6 -0.3]
 [-0.2  0.5]
 [ 0.4  0.7]]
Bias: [0.1, -0.8]
Weighted Sum (Z2): [[ 6.211 -2.465]]
Activation (Sigmoid): [[0.997997 0.078327]]
Note: weights_2 were transposed to match input shape.

Layer 3 (Output Layer)
Inputs: [[0.997997 0.078327]]
Weights (shape (2, 1)):
[[ 0.7]
 [-0.5]]
Bias: [0.2]
Weighted Sum (Z3): [[0.859]]
Activation (Sigmoid): [[0.702542]]
Note: weights_3 were transposed to match input shape.

Final Classification Result
Predicted Probability (sigmoid): [[0.702542]]
Ta